In [1]:
import pandas as pd
import plotly.express as px
from dash import dcc, html, Input, Output
from jupyter_dash import JupyterDash

# Load your CSV file
df = pd.read_csv('docs/owid-covid-data.csv')

# Select relevant columns
country_code_column = 'iso_code'
gdp_per_capita_column = 'gdp_per_capita'
vaccinations_column = 'total_vaccinations_per_hundred'

# Extract relevant data
data = df[[country_code_column, gdp_per_capita_column, vaccinations_column]].copy()

# Ensure country codes are in uppercase (Plotly expects uppercase ISO codes)
data['country_code'] = data[country_code_column].str.upper()

# Drop rows where GDP per capita or vaccinations per hundred is NaN
data.dropna(subset=[gdp_per_capita_column, vaccinations_column], inplace=True)

# Define a custom color scale from light blue to deep blue
custom_color_scale = [
    [0, 'rgb(173, 216, 230)'],  # light blue
    [1, 'rgb(0, 0, 139)']       # deep blue
]

# Initialize the Dash app
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='metric-dropdown',
        options=[
            {'label': 'GDP per Capita', 'value': gdp_per_capita_column},
            {'label': 'Vaccinations per 100 Inhabitants', 'value': vaccinations_column}
        ],
        value=gdp_per_capita_column,
        clearable=False
    ),
    dcc.Graph(id='choropleth-map')
])

@app.callback(
    Output('choropleth-map', 'figure'),
    Input('metric-dropdown', 'value')
)
def update_choropleth(selected_metric):
    fig = px.choropleth(
        data, 
        locations='country_code', 
        color=selected_metric,
        hover_name='country_code',
        hover_data={gdp_per_capita_column: True, vaccinations_column: True},
        title=f'{selected_metric.replace("_", " ").title()} for 2021',
        labels={selected_metric: selected_metric.replace("_", " ").title()},
        color_continuous_scale=custom_color_scale,
        range_color=(0, 60000) if selected_metric == gdp_per_capita_column else (0, 100)
    )
    return fig

# Run the app in the notebook
app.run_server(mode='inline')

/opt/homebrew/lib/python3.12/site-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

